In [15]:
# use the import keyword to import pandas, requests, and bs4 modules
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

In [16]:
# assign the NY WARN notice url to a variable
url = "https://labor.ny.gov/app/warn/"
url

'https://labor.ny.gov/app/warn/'

In [17]:
# define headers
headers = {'accept-encoding': 'deflate'}

In [20]:
# make a get request to the url using the requests library and assign the response to a variable called 'response'
response = requests.get(url, headers=headers)

In [21]:
# print out status code of response to confirm that your request worked
response.status_code

200

In [22]:
# parse the response text using Beautiful Soup's html parser and assign output to a variable called 'soup'
# response.text
type(response.text)

str

In [23]:
# parse the response text using Beautiful Soup's html parser and assign output to a variable called 'soup'
soup = BeautifulSoup(response.text, 'html.parser')

In [24]:
# scrape the first table on the page and assign it to a variable called 'table'
table = soup.find("table")

In [25]:
# grab all rows from the table and assign to a variable called 'rows'
rows = soup.find_all("tr")

In [26]:
# print out the number of rows — this is how many WARN notices there were in 2020
len(rows)

1243

In [38]:
# make an array called 'results'
results = []

# loop through the rows using a for loop
for row in rows:
    
    # grab the anchor tag (the link tag) in the row and then grab the href attribute from the tag
    a = row.find("a")['href']
    
    # concatenate the root url from above with this href attribute and assign to a variable called 'company_url'
    company_url = f'{url}{a}'
    print(company_url)
    
    # make a get request to the company url assign the response to a variable called 'company_response'
    company_response = requests.get(company_url, headers=headers)
    
    # parse the response text and assign output to a variable called 'company_soup'
    company_soup = BeautifulSoup(company_response.text, 'html.parser')
    
    # grab the first table on the page
    company_table = company_soup.find('table')
    
        # unwrap all of the spans
    
    
    # loop through all of the p tags
    paragraphs = company_table.find_all("p")
    for p in paragraphs:
        # grab all of the values we want
        text = p.get_text('\n')
        if 'Date of Notice:' in text:
            notice_date = text.split(":")[1].strip().split('\n')[0].strip().replace(',', '').replace(';', '')
            # print(notice_date)
        elif 'Reason Stated for Filing:' in text:
            reason = text.split(":")[1].strip()
            # print(reason)
        elif 'Company:' in text:
            split_company = text.split('\n')
            company = split_company[1].strip()
            # print(split_company)
            address = ''.join(split_company[2:])
            #print(company)
            #print(address)
        elif 'County:' in text:
            county = f'{text.split(":")[1].strip().split("|")[0].strip()} County'
            #print(county)
        elif 'Phone:' in text:
            phone = text.split(":")[1].strip()
            #print(phone)
        elif ('Business Type:' in text):
            business_type = text.split(":")[1].strip()
            #print(business_type)
        elif ('Number Affected:' in text):
            affected = text.split(":")[1].strip().split()[0].replace(',', '')
            #print(affected)
        elif ('Total Employees:' in text):
            if "------" in text:
                total_employees = ''
            else:
                total_employees = text.split(":")[1].strip().split(" ")[0].strip().replace(',', '')
                #print(total_employees)
        elif ('Layoff Date:' in text):
            layoff_date = text.split(":")[1].strip().split(' ')[0].strip()
            #print(layoff_date)
        elif ('Reason for Dislocation:' in text):
            dislocation = text.split(":")[1].strip()
            #print(dislocation)
        elif ('Union:' in text):
            union = text.split(":")[1].strip()
            #print(union)
        elif ('Classification:' in text):
            classification = text.split(":")[1].strip()
            #print(classification)
            
    # store values in a result object
    result = {
        'notice_date': notice_date,
        'reason': reason,
        'company': company,
        'address': address,
        'phone': phone,
        'business_type': business_type,
        'affected': affected,
        'total_employees': total_employees,
        'layoff_date': layoff_date,
        'dislocation': dislocation,
        'union': union,
        'classification': classification
     }

    # append result object to results
    results.append(result)
    
    break
    #remove break to have the loop run through for each paragraph

https://labor.ny.gov/app/warn/details.asp?id=8305


In [62]:
# wrap results in a dataframe
df = pd.DataFrame(results)
df

,notice_date,reason,company,address,phone,business_type,affected,total_employees,layoff_date,dislocation,union,classification
0,4/17/2020,Temporary Plant Layoff,Wolf Eastside LLC dba Wolfgang's Steakhouse,"200 East 54th Street New York, NY 10012",(646) 642-4655,Restaurant,65,-----,3/19/2020,Unforeseeable business circumstances prompted ...,The employees are not represented by a union,Temporary Plant Layoff
1,4/21/2020,Plant Layoff,Rising Ground,Family Treatment and Rehabilitation Program an...,(914) 375-8758,Human Services,54,-----,5/29/2020,Contract Loss,The employees are not represented by a union,Plant Layoff
2,4/27/2020,Plant Closing,"Spirit & Sanzone Distributors Co., Inc.","6495 Fly Road, P.O. Box 696 East Syracuse, NY ...",(315) 463-6103,Wine Wholesaler,64,-----,6/1/2020,Sale of Business,The employees are not represented by a union,Plant Closing
3,4/27/2020,Plant Closing,"Other Spirit & Sanzone Distributors Co., Inc.","9396 Route 49 Marcy, NY 13403",(315) 463-6103,Wine Wholesaler,32,,6/1/2020,Sale of Business,The employees are not represented by a union,Plant Closing
4,3/18/2020,Temporary Plant Closing,"Birth of the Cool, LLC dba Eleven Madison Park","11 Madison Ave. New York, NY 10010",(647) 747-2612,Restaurant,178,178,3/18/2020,Unforeseeable business circumstances prompted ...,The employees are not represented by a union.,Temporary Plant Closing
...,...,...,...,...,...,...,...,...,...,...,...,...
1232,1/6/2020,Plant Closing,Macy's Broadway Mall Store (Macy's Retail Hold...,"100 Broadway Mall Hicksville, NY 11801",(646) 429-7462,Retail Store,155,155,Macy's,Economic,The employees are not represented by a union.,Plant Closing
1233,12/27/2019,Temporary Plant Closing,Wesley Gardens Nursing Home,"3 Upton Park Rochester, NY 14607",(585) 241-2105,Nursing Home,132,132,Beginning,Due to a water line break,1199 SEIU,Temporary Plant Closing
1234,12/30/2019,Plant Closing,"127 W. 43rd St. Chophouse, Inc. (Heartland Bre...","127 West 43rd StreetNew York, NY 10018",(917) 999-6532,Restaurant,106,106,The,Economic,The employees are not represented by a union.,Plant Closing
1235,12/30/2019,Plant Closing,"New York Express and Logistics, LLC","292 Wolf Road Latham, NY 12110",(617) 968-5311,Trucking company providing freight transportat...,48,48,3/31/2020,Contract between New York Express and Logistic...,The employees are not represented by a union.,Plant Closing


In [10]:
# output dataframe to a csv
df.to_csv('data/warn.csv', index=False)